In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

In [2]:
%matplotlib

Using matplotlib backend: Qt5Agg


# Advección

$$
\frac{\partial \psi}{\partial t} + \nabla\cdot\left(\psi\vec{u}\right) = 0
$$

# Lax Wendroff
$$
\frac{\partial \psi}{\partial t} + \frac{\partial f(\psi)}{\partial x} = 0
$$

$$
\psi_i^{n+1} = \psi_i^{n} - \frac{\Delta t}{2\Delta x}A[\psi_{i+1}^{n}-\psi_{i-1}^{n}] + \frac{\Delta t^2}{2\Delta x^2}A^2[\psi_{i+1}^{n}-2\psi_i^n+\psi_{i-1}^{n}]
$$

## Convention
$$
\begin{matrix}
f(\psi) & = & u_x\psi \\
A & = & u_x
\end{matrix}
$$

In [3]:
def lax_wendroff_step(actual: np.array, alpha: float, beta: float) -> np.array:
    # TODO: boundary conditions
    actual[1:-1] = actual[1:-1] - alpha * (actual[2:] - actual[:-2]) + beta * (actual[2:] - 2 * actual[1:-1] + actual[:-2])
    return actual

def lax_wendroff(psi: np.array, dt: float, dx: float, speed: float, stop_time: float, store_every: int = 1):
    if dx / dt < speed:
        raise(ValueError(f"Speed {speed} is greater than information propagation speed {dx / dt}"))
        
    alpha = dt / dx * 0.5 * speed
    beta = 0.5 * (dt * speed / dx) ** 2
    
    output = []
    psi = psi.copy()
        
    for i in range(int(stop_time // dt)):
        if i % store_every == 0:
            output.append(psi.copy())
            
        psi = lax_wendroff_step(psi, alpha, beta)
        
    return np.vstack(output)

In [4]:
n = 1000
x = np.linspace(0, 1, n)
psi = np.zeros(n)
psi[int(n / 2 * (0.95)) : int(n / 2 * (1.05))] = 1

dt = 1e-2

In [5]:
output = lax_wendroff(psi, dt=dt, dx=x[1]-x[0], speed=0.1, stop_time=200 * dt, store_every=5)

In [6]:
fig, ax = plt.subplots()

line, = ax.plot(x, psi)

def anim(i):
    global line, x, output
    line.set_data(x, output[i])
    return line,

anim = FuncAnimation(
    fig,
    anim,
    frames=len(output),
    interval=len(output) / 24
)

### Energía

$$ k = \frac{1}{2}u^2 $$
$$ e = E - k $$

### Presión

$$ P = \rho e (\gamma - 1)  $$

### h

$$ h = e + \frac{P}{\rho} $$

### $c_s$

$$ c_s = h(\gamma + 1) $$

### Sistema de ecuaciones

$$ \frac{\partial}{\partial t} 
\begin{pmatrix}
\rho \\
\rho u \\
\rho E 
\end{pmatrix} + 
\frac{\partial}{\partial x}
\begin{pmatrix}
\rho u\\
\rho u^2+P \\
\rho u(E + P/\rho)
\end{pmatrix}
= 0$$

In [10]:
gamma = 3/2
delta_t = 0.1
delta_x = 0.1

In [8]:
def e(u):
    return u[2] - 0.5 * (u[1] ** 2) / u[0]

def p(u):
    return u[0] * e[u] * (gamma - 1)

In [11]:
def F_u(u):
    return np.array([u[1], (u[1] ** 2)/u[0] + p(u), u[1] * U(u[2] + p(u)) / u[0]])

In [12]:
def lax_step(U):
    U_pos_h = 0.5*(U[:, 2:] + U[:, 1:-1]) -  0.5*delta_t * (np.apply_along_axis(F_u, 0, U[:, 2:]) - np.apply_along_axis(F_u, 0, U[:, 1:-1])) / delta_x
    U_neg_h = 0.5*(U[:, 1:-1] + U[:, :-2]) -  0.5*delta_t * (np.apply_along_axis(F_u, 0, U[:, 1:-1]) - np.apply_along_axis(F_u, 0, U[:, :-2])) / delta_x
    U_adv = U[1:-1] - delta_t * (np.apply_along_axis(F_u, 0, U_pos_h) - np.apply_along_axis(F_u, 0, U_neg_h))
    return U_adv